## Instagram Scraper

In [62]:
# import dependencies 
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [139]:
yelp_museum_df = pd.read_csv('yelp_museums.csv')
yelp_museum_df.reset_index(inplace=True, drop=True)
yelp_museum_df[['museum names', 'museum names no space', 'museum ratings', 'number of museum reviews']]

,museum names,museum names no space,museum ratings,number of museum reviews
0,Smithsonian National Museum of Natural History,SmithsonianNationalMuseumofNaturalHistory,4.5,1002
1,Newseum,Newseum,4.5,920
2,Smithsonian National Air and Space Museum,SmithsonianNationalAirandSpaceMuseum,4.0,1066
3,National Gallery of Art,NationalGalleryofArt,4.5,653
4,National Portrait Gallery,NationalPortraitGallery,4.5,553
5,National Museum of American History,NationalMuseumofAmericanHistory,4.0,629
6,National Museum of African American History an...,NationalMuseumofAfricanAmericanHistoryandCulture,4.5,763
7,United States Holocaust Memorial Museum,UnitedStatesHolocaustMemorialMuseum,4.5,755
8,United States Botanic Garden,UnitedStatesBotanicGarden,4.5,456
9,Smithsonian American Art Museum Renwick Gallery,SmithsonianAmericanArtMuseumRenwickGallery,4.5,409


In [81]:
# get each individual instagram url
hashtags = yelp_museum_df['museum names no space']
browser = webdriver.Chrome('<ENTER_DRIVER_PATH>')

# create empty lists to append later 
links = []
locations = []

# loop through each of the hashtags 
for hashtag in hashtags:
    browser.get('https://www.instagram.com/explore/tags/' + hashtag)
    
    # parse html source
    source = browser.page_source
    data = bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    
    # loop through the links and generate the unique url for each ig post
    for link in script.findAll('a'):
        if re.match("/p", link.get('href')):
            links.append('https://www.instagram.com' + link.get('href'))
            locations.append(hashtag)
            
    Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [112]:
# parse out the number of comments and likes

# create empty lists to append to later
num_comments = []
num_likes = []

for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data = bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data = json.loads(raw)
        posts = json_data['entry_data']['PostPage'][0]['graphql']
        
        # parse out the number of comments for each post
        comments = posts['shortcode_media']['edge_media_to_parent_comment']['count']
        num_comments.append(comments)
        
        # parse out the number of likes for each post
        likes = posts['shortcode_media']['edge_media_preview_like']['count']
        num_likes.append(likes)
       
    except:
        np.nan

In [113]:
# do not want to grab the first ig link since it is the username...
# when i do this it cuts off the last one too for some reasonnnnnn
print(len(links))
print(len(num_likes))
print(len(num_comments))
print(len(locations))

153
139
139
153


In [121]:
insta_df

,location,links,number of comments,number of likes
0,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1R3wjLnX5z/,4,24
1,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1UftUGnXA8/,2,44
2,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1YrcFLgIQv/,1,31
3,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/BqjVRMyH0mG/,2,12
4,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/By_GQZMhEbS/,2,14
5,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/By-Pz2OhEvK/,14,124
6,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B0mW9QIJ21A/,1,192
7,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B1Z5NUchXuM/,2,13
8,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/By6qlD5hP1_/,0,22
9,SmithsonianNationalMuseumofNaturalHistory,https://www.instagram.com/p/B17uVecnMtK/,0,13


In [119]:
# export df as a csv file
insta_df.to_csv(r'ig_data.csv')

In [137]:
# get the sum of the numnber of likes and number of comments grouped by location
grouped_insta_df = insta_df.groupby(['location'])['number of likes', 'number of comments'].sum()

In [138]:
# export grouped df as a csv
grouped_insta_df.to_csv(r'ig_data_summary.csv')